<a href="https://colab.research.google.com/github/peternguyen24/NeuralNetworkBasic/blob/main/language/wavenet_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Video: https://www.youtube.com/watch?v=t3YJ5hKiMQ0&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=6

In [1]:
 words = open('/content/drive/MyDrive/Colab Notebooks/names.txt', 'r').read().splitlines()

In [2]:
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

#Part 1: Dataset

In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [4]:
chars = set()
for word in words:
  for ch in word:
    chars.add(ch)
chars = sorted(list(chars))

stoi = {ch: idx+1 for idx, ch in enumerate(chars)}
stoi['.'] = 0

itos = {idx: ch for ch, idx in stoi.items()}
print(itos)
print(stoi)

VOCAB_SIZE = len(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}


In [5]:
# BLOCK_SIZE = 3
# X, Y  = [], []

# for w in words:
#   # print(w)
#   context = [0] * BLOCK_SIZE
#   for ch in w + '.':
#     ix = stoi[ch]
#     X.append(context)
#     Y.append(ix)
#     # print(''.join(itos[i] for i in context), '--->', itos[ix])
#     context = context[1:] + [ix]

# X = torch.tensor(X)
# Y = torch.tensor(Y)
# data_size = X.nelement()

In [6]:
# X.shape, X.dtype, Y.shape, Y.dtype


In [7]:
# training split, validation split, test split
# 80% 10% 10%

BLOCK_SIZE = 8 # Take 8 character to build next char

def build_dataset(words):
  X, Y  = [], []
  for w in words:
  # print(w)
    context = [0] * BLOCK_SIZE
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      # print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix]
  return torch.tensor(X), torch.tensor(Y)

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

print(Xtr.shape, Ytr.shape)
print(Xdev.shape, Ydev.shape)
print(Xte.shape, Yte.shape)


torch.Size([182625, 8]) torch.Size([182625])
torch.Size([22655, 8]) torch.Size([22655])
torch.Size([22866, 8]) torch.Size([22866])


# Training


In [24]:
# Initialization

class Linear:
  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5 # ensure uniform distribution instead of gaussian
    self.bias = torch.zeros(fan_out) if bias else None

  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out

  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])

class BatchNorm1d:
  def __init__(self, dim, eps=1e-5, momentum=0.01):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)

  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      if x.ndim == 2:
        dim = 0
      elif x.ndim == 3:
        dim = (0,1)
      xmean = x.mean(dim, keepdim=True) # batch mean
      xvar = x.var(dim, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]


class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out

  def parameters(self):
    return []


class Embedding:
  def __init__(self, num_embeddings, embedding_dim):
    self.weight = torch.randn((num_embeddings, embedding_dim))

  def __call__(self, IX):
    self.out = self.weight[IX]
    return self.out

  def parameters(self):
    return [self.weight]

class FlattenConsecutive:
  def __init__(self,n):
    self.n = n
  def __call__(self, x):
    B, T, C = x.shape
    x = x.view(B, T // self.n, C * self.n)
    if x.shape[1] == 1:
      x = x.squeeze(1)
    self.out = x
    return self.out

  def parameters(self):
    return []

class Sequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    self.out = x
    return self.out

  def parameters(self):
    return [p for layer in self.layers for p  in layer.parameters()]



In [25]:
torch.manual_seed(42);

Old
* Embedding : (4, 8, 10)
* FlattenConsecutive : (4, 80)
* Linear : (4, 200)
* BatchNorm1d : (4, 200)
* Tanh : (4, 200)
* Linear : (4, 27)

New
* Embedding : (4, 8, 10)
-> (4, 8, 10)
-> (4, 4, 20)
-> (4, 2, 40)
-> (4, 1, 80) -> (4,40)

* FlattenConsecutive : (4, 80)
* Linear : (4, 200)
* BatchNorm1d : (4, 200)
* Tanh : (4, 200)
* Linear : (4, 27)

In [26]:
n_embd = 10 # the dementialnaity of the char embedding vectors
n_hidden = 68 # the number of neurons in the hidden layer of the MLP

model = Sequential([
          Embedding(VOCAB_SIZE, n_embd),
          FlattenConsecutive(2), Linear(n_embd * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
          FlattenConsecutive(2), Linear(n_hidden * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
          FlattenConsecutive(2), Linear(n_hidden * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
          Linear(n_hidden, VOCAB_SIZE)])


with torch.no_grad():
  model.layers[-1].weight *= 0.1 # gain - make last layer less confident

parameters = model.parameters()
for p in parameters:
  p.requires_grad = True
print(sum(p.nelement() for p in parameters))

22397


In [27]:
# Optimization

MAX_STEPS = 200000
BATCH_SIZE = 32
lossi = []

for i in range(MAX_STEPS):
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (BATCH_SIZE,)) # Get the random indexes of 32 words from training data. shape = (32)
  Xb, Yb = Xtr[ix], Ytr[ix] # Get the actual (32, BLOCK_SIZE)

  # forward
  logits = model(Xb)
  loss = F.cross_entropy(logits, Yb)

  # backward
  for p in parameters:
    p.grad = None
  loss.backward()

  # update
  lr = 0.1 if i < 150000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 10000 == 0:
    print(f'{i:7d}/{MAX_STEPS:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())

  # break # for debugging

      0/ 200000: 3.3142
  10000/ 200000: 2.2096
  20000/ 200000: 2.1495
  30000/ 200000: 2.1232
  40000/ 200000: 1.9629
  50000/ 200000: 2.4183
  60000/ 200000: 1.9633
  70000/ 200000: 1.8960
  80000/ 200000: 2.1884
  90000/ 200000: 1.8161
 100000/ 200000: 1.5931
 110000/ 200000: 2.3607
 120000/ 200000: 2.3670
 130000/ 200000: 1.9098
 140000/ 200000: 2.2285
 150000/ 200000: 1.7118
 160000/ 200000: 1.8517
 170000/ 200000: 1.9997
 180000/ 200000: 2.1043
 190000/ 200000: 1.6968


In [28]:
for layer in model.layers:
  print(layer.__class__.__name__, ':', tuple(layer.out.shape))

Embedding : (32, 8, 10)
FlattenConsecutive : (32, 4, 20)
Linear : (32, 4, 68)
BatchNorm1d : (32, 4, 68)
Tanh : (32, 4, 68)
FlattenConsecutive : (32, 2, 136)
Linear : (32, 2, 68)
BatchNorm1d : (32, 2, 68)
Tanh : (32, 2, 68)
FlattenConsecutive : (32, 136)
Linear : (32, 68)
BatchNorm1d : (32, 68)
Tanh : (32, 68)
Linear : (32, 27)


In [22]:
# plt.plot(torch.tensor(lossi).view(-1, 1000).mean(1))

In [29]:
for layer in model.layers:
  layer.training = False
# evaluate the loss
@torch.no_grad()
def split_loss(split):
  x, y = {
      'train': (Xtr, Ytr),
      'val': (Xdev, Ydev),
      'test': (Xte, Yte),
  }[split]
  logits = model(x)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')
# split_loss('test')

train 1.909277081489563
val 2.0185766220092773


# Performance log

* (3 char, 200 hidden neurons, 12k params): Train 2.08, val 2.105
* (8 char, 22K params): train 1.905, val 2.048
* (8 char structured, 22k params - 68 hidden layer): train 1.94, val 2.02
* (8 char structured, 22k params) with batchnorm fixed: train 1.909, val 2.0186

In [ ]:
# sample from the model

for _ in range(20):
  out = []
  context = [0] * BLOCK_SIZE # initialize with all ...
  while True:
    # forward
    logits = model(torch.tensor([context]))
    probs = F.softmax(logits, dim=1)
    # sample from the distribution
    ix = torch.multinomial(probs, num_samples=1).item()

    # shift the context windows and track the samples
    context = context[1:] + [ix]
    out.append(ix)

    # if we sample the special '.' token, break
    if ix == 0:
      break

  print(''.join(itos[i] for i in out)) # decode and print

Takeaway:
* This wave net structure is for efficient rather than actual better understanding
* The grouping is actually similar to sliding. Instead of sliding to each individual case, it takes some composition into kernel of NN.